Analysis of integrative models<a id="mainpage"></a>
================================

**Table of contents**

 - [Introduction](#intro)
 - [Get precalculated outputs](#getoutputs)
 - [Find good scoring models](#filtermodels)
 - [Determining sampling precision, clustering, and computing localization densities](#sampprec)
  - [Procedure](#exhaustproc)
  - [Running the script](#exhaustrun)
  - [Results](#results)
  - [Discussion](#exhaustdiscuss)
 - [Fit to data](#fitdata)
 - [Resampling](#resampling)
 - [Further reading](#further)


# Introduction<a id="intro"></a>

In this tutorial we will look at the output from the previous tutorial, modeling with cross-links and EM data.

Once we have generated a number of candidate models from our sampling run(s), analysis consists of a number of steps (below).

<img src="images/validation_pipeline.png" width=600px title="Validation pipeline" />

First, we select a subset of the models for analysis, perhaps by discarding models that violate too many of the restraints or from early in each run before equilibration is complete.

The next step is to assess whether our sampling was exhaustive (more on this below). If we have not sampled the scoring function surface adequately at some level of resolution, we cannot trust our models. In broad strokes, this is done by taking two independent samples and comparing them; if sampling is exhaustive they should sample the same conformational space to the same degree.

Two more indications of model quality are a good fit to the data that were used to construct the restraints, plus a good fit to data that was not used in the modeling (for example data that were intentionally held back, or that are difficult or expensive to use as restraints).

In certain situations we can also estimate the variance of the input data by resampling. For example, when using cross-linking data we can repeat the entire modeling procedure leaving out a random fraction of the cross-links.

This tutorial largely follows the protocol described in more detail in [Viswanath *et al.*, "Assessing Exhaustiveness of Stochastic Sampling for Integrative Modeling of Macromolecular Structures". Biophys J **113**, 2344-2353, 2017](https://www.ncbi.nlm.nih.gov/pubmed/29211988).

# Get precalculated outputs<a id="getoutputs"></a>

First, download `imp_tutorial_pol3_xl_cryoem_premodeling.v1.1.tar.gz` from [Zenodo](https://doi.org/10.5281/zenodo.3523241) and extract it in the same directory (`analysis`) as this notebook. (This should already have been done for you.)

This file contains RMF and stat file outputs for longer Monte Carlo runs using the previously-demonstrated script, with multiple replicas:

In [ ]:
ls imp_tutorial_pol3_xl_cryoem_premodeling/modeling/

Note that there are three output directories. This is because the protocol was run three times, each time with a better approximation to the EM map (more GMMs). For more information, see [Bonomi *et al.*, "Bayesian weighing of electron cryo-microscopy data for integrative structural modeling". Structure, pii: S0969-2126(18)30337-X, 2018](https://www.ncbi.nlm.nih.gov/pubmed/30393052).

For the purposes of this tutorial we'll look at the final (shortest) trajectory, `A_output_3`:

In [ ]:
ls imp_tutorial_pol3_xl_cryoem_premodeling/modeling/A_output_3/rmfs/

# Find good scoring models<a id="filtermodels"></a>



The scripts for this protocol assume that the outputs are stored in a particular directory structure, i.e. `<top directory>/<run number>/output/`, so we first need to make a symlink to the precalculated outputs:

In [ ]:
mkdir -p modeling/run1
ln -sf ../../imp_tutorial_pol3_xl_cryoem_premodeling/modeling/A_output_3 \
       modeling/run1/output

The `select_good_scoring_models.py` script filters models based on score and parameter thresholds, i.e. it will examine all of the models from all frames and all replicas, and potentially over multiple runs (although we have only a single run in this case).

In this script, required flags are:
 - `-rd`, which specifies the directory containing sampling output folders
   (`modeling` in this case, to match the symlink we created above);
 - `-rp`, which defines the prefix for the sampling output folders (`run` in this case as per the symlink);
 - `-sl`, which defines the stat file keywords (see below) that we wish to filter on;
 - `-pl`, which specifies the keywords that will be written to the output file;
 - `-alt` and `-aut`, which specify, respectively, the lower and upper threshold for each keyword in `-slt` hat define acceptance.

The `-mlt` and `-mut` keywords, which are optional, define thresholds for restraints made
of multiple components (such as crosslinks).

Note that a list of acceptable stat file keywords can be determined using the `scripts/show_stat.py` script (or by looking at a stat file in a text editor, as they have a very simple format):

In [ ]:
python scripts/show_stat.py modeling/run1/output/stat.0.out \
       |grep -v MonteCarlo | cut -f1 -d\| | uniq

Here, we first use crosslink satisfaction as an initial filtering criterion because we usually have an *a priori* estimate of the false positive rate and/or cutoff distance (for scores whose thresholds are not known *a priori*,
one can perform a multi-stage filtering process as outlined in the above protocol). Here, we only accept models with at least 90% of the cross-links satisfied by setting `-alt` to 0.9 and `-aut` to 1.0. A crosslink is considered satisfied if the distance is between 0.0 and 30.0 Å, as delineated by the `-mlt` and `-mut` keywords, respectively. We ask that connectivity, crosslink data score, excluded volume, EM, and total scores be printed as well.

This script can be run from the cell below:

In [ ]:
python scripts/select_good_scoring_models.py \
    -rd modeling -rp run \
    -sl CrossLinkingMassSpectrometryRestraint_Distance_ \
    -pl ConnectivityRestraint_ABC10alpha \
        CrossLinkingMassSpectrometryRestraint_Data_Score_XL \
        ExcludedVolumeSphere_None \
        GaussianEMRestraint_Total \
        Total_Score \
    -alt 0.9 -aut 1.0 -mlt 0.0 -mut 30.0

This script creates a directory `filter` and a file,
`filter/models_scores_ids.txt`, that contains the model index, its run,
replicaID, frame ID, scores, and sample ID for each model:

In [ ]:
head filter/model_ids_scores.txt

We can now use the script `plot_score.py` to plot the distribution of the EM, connectivity
and excluded volume scores from this first set of filtered models to
determine a reasonable threshold for accepting or rejecting a model.

In [ ]:
python scripts/plot_score.py \
    filter/model_ids_scores.txt \
    GaussianEMRestraint_Total

In [ ]:
python scripts/plot_score.py \
    filter/model_ids_scores.txt \
    ConnectivityRestraint_ABC10alpha

In [ ]:
python scripts/plot_score.py \
    filter/model_ids_scores.txt \
    ExcludedVolumeSphere_None

<img src="ExcludedVolumeSphere_None.jpg" width=300px title="Connectivity score distribution" /> <img src="ConnectivityRestraint_ABC10alpha.jpg" width=300px title="Connectivity score distribution" /> <img src="GaussianEMRestraint_Total.jpg" width=300px title="EM score distribution" />

The resulting histograms are roughly Gaussian. Based on these distributions we can set our criteria for good scoring models, generally to some value below the mean (e.g. 1 standard deviation) except for connectivity which appears well satisfied in almost all models. For this demonstration we filter on the EM and excluded volume scores, with high score thresholds of 5450 and 90 respectively.

Note that currently, the choice of filtering criteria is very subjective. Another approach is to take all models after some statistically-determined "burn in" or equilibration period. See https://github.com/salilab/PMI_analysis for a method in development to do just this, and [Chodera, "A Simple Method for Automated Equilibration Detection in Molecular Simulations", J Chem Theory Comput **12**, 1799-1805, 2016](https://doi.org/10.1021/acs.jctc.5b00784) for a description of the algorithm.

We rerun `select_good_scoring_models.py` adding the extra keywords and score thresholds. These thresholds return 1671 good scoring models.

> In general, we require at least 1000 or more models for assessing sampling exhaustiveness. Our score thresholds were chosen in order to have a reasonable number (1000-20000) models for analysis. If we have too few models, the satisfaction criteria should be relaxed, or more sampling should be performed to find more satisfactory models. Too many models (\>20,000) will make subsequent processing more computationally intensive; in this case satisfaction criteria can be made stricter, or one can pass a random subset of these models to the sampling convergence protocol.

In [ ]:
python scripts/select_good_scoring_models.py \
    -rd modeling -rp run \
    -sl CrossLinkingMassSpectrometryRestraint_Distance_ \
        GaussianEMRestraint_Total \
        ExcludedVolumeSphere_None \
    -pl ConnectivityRestraint_ABC10alpha \
        CrossLinkingMassSpectrometryRestraint_Data_Score_XL \
        Total_Score \
    -alt 0.9 -50 -50 -aut 1.0 5450 90 \
    -mlt 0.0 0.0 0.0 -mut 30.0 0.0 0.0
    
wc -l filter/model_ids_scores.txt

# Determining sampling precision, clustering, and computing localization densities<a id="sampprec"></a>

The `Master_Sampling_Exhaustiveness_Analysis.py` script is used to
calculate the sampling precision of the modeling. During this step,
multiple tests for convergence are performed on the two samples
determined above, models are clustered, and localization densities are computed.

## Procedure<a id="exhaustproc"></a>

Ideally, we want to generate the complete ensemble of models consistent with the input information (good-scoring models). The variation among the models in this ensemble quantifies the uncertainty of modeling (**model precision**).

We need to know the model precision because

 - it gives an estimate of the aggregate uncertainty in the input information;
 - it likely provides the lower bound on model accuracy;
 - applications of models strongly depend on their accuracy;
 - only when it is known can the model be used to inform future choices, such as whether to gather more data, change the system representation, scoring functions, or sampling algorithms.

Generally, it is not possible to sample completely because the space is just too large, so we have to use stochastic sampling such as Monte Carlo, and we can only aim to find *representative* good-scoring models.

These representative models sample all good-scoring models at some precision, which we define as the **sampling precision**. Clearly, the sampling precision imposes a lower limit on the model precision.

Therefore, exhaustive sampling of good-scoring models is a prerequisite for accurate modeling and assessment of model precision.

Sampling is *exhaustive* at a certain precision when it generates all sufficiently good-scoring models at this precision.

> Sampling exhaustiveness and sampling precision are invariably intertwined. There is always a precision at which any sampling is exhaustive; for example, even a single structure provides an exhaustive sample at a precision worse than the scale of the system.

As a proxy for assessing sampling exhaustiveness, we evaluate whether or not two independently and stochastically generated sets of models (model samples) are sufficiently similar. Passing these tests is a necessary, but not sufficient condition for exhaustive sampling. (For example, part of the sampling space could be missed in all samples.)

Four tests must be passed as part of the protocol:

 - convergence of the model score;
 - whether model scores for the two model samples were drawn from the same parent distribution;
 - whether each structural cluster includes models from each sample proportionally to its size;
 - whether there is sufficient structural similarity between the two model samples in each cluster.
 
The tests are evaluated using the following steps:
 
<img src="images/exhaust-flowchart.png" width=300px title="Sampling exhaustiveness flowchart" />

The distance threshold-based clustering merits further explanation. We define the sampling precision as the radius of the clusters in the finest clustering for which

 - each sample contributes models proportionally to its size (considering both significance and magnitude of the difference)
 - a sufficient proportion of all models occur in sufficiently large clusters
 
This is demonstrated below in 2D space. Two independent equal-sized random samples of good-scoring models are shown in red and blue. Models in the two samples are clustered together. The gray circles indicate cluster boundaries and the gray-scale indicates the density of models in the cluster. The size of the circles indicates the clustering threshold. The test assesses whether the proportion of models from the two samples (red and blue) is similar in each significant cluster. Note that some models are shown as open circles, indicating that these models belong to insignificant clusters (i.e., small clusters containing few models).
 
<img src="images/chi2-test.png" width=400px title="χ² test" />

## Running the script<a id="exhaustrun"></a>

The script first needs a file, `density_ranges.txt`, that defines components
using PMI selection tuples on which we calculate localization densities.
Here, we ask for five localization densities for the C53, C37, C82, C34 and C31 subunits of the complex.

In [ ]:
cat <<END > density_ranges.txt
density_custom_ranges={
    'C53':['C53'],
    'C37':['C37'],
    'C82':['C82'],
    'C34':['C34'],
    'C31':['C31'] }
END

Note that the exhaustiveness test scripts take some time (~2 hours) to run. Thus, it is not recommended to run these scripts during the tutorial - they are presented here for completeness. Below, we show sample outputs.

First, we rerun the selection of good scoring models script, adding the `-e` flag. This will select models as before; additionally, it will extract coordinates for these models as RMF files, placing them in the `good_scoring_models` directory. It will divide the models into two approximately-equal-sized samples, A and B. If we had multiple independent runs, it would simply choose models from different runs for the different samples. In this case, since we have only a single run, it will divide it into two random subsets.

⚠️ This cell takes about 30 minutes to run!

In [ ]:
python scripts/select_good_scoring_models.py \
    -rd modeling -rp run \
    -sl CrossLinkingMassSpectrometryRestraint_Distance_ \
        GaussianEMRestraint_Total \
        ExcludedVolumeSphere_None \
    -pl ConnectivityRestraint_ABC10alpha \
        CrossLinkingMassSpectrometryRestraint_Data_Score_XL \
        Total_Score \
    -alt 0.9 -50 -50 -aut 1.0 5450 90 \
    -mlt 0.0 0.0 0.0 -mut 30.0 0.0 0.0 -e

We now run the script for testing sampling exhaustiveness. This takes a number of arguments:

 - `-n` defines the labels (`rnapoliii`) for the output files.
 - `-a` aligns all models before calculating pairwise RMSD (this may not be necessary when models are already aligned, for example when they are already aligned to a good quality EM map).
 - `-g` determines the step size in Å for calculating sampling precision. (This is the step size at which clustering is performed between the minimum and maximum RMSDs in the dataset. Here we use 0.1Å to get a very precise estimate of the sampling precision; however this results in a very long calculation. In practice, especially for larger systems whose sampling precision will be much lower, one would choose a larger value to make calculation more efficient.)
 - `-m cpu_omp` and `-c N` run the RMSD calculation in parallel, where N is the number of processors. (The GPU mode of pyRMSD generally increases performance significantly. It is invoked by using `-m cuda`.)
 - `-p` defines the path to the good scoring model directory.
 - `-gp` will generate plots using `gnuplot`.

⚠️ This cell takes about 90 minutes to run!

In [ ]:
python scripts/Master_Sampling_Exhaustiveness_Analysis.py \
       -n rnapoliii -p good_scoring_models/ \
       -d density_ranges.txt -m cpu_omp -c 8 \
       -a -g 0.1 -gp

## Results<a id="results"></a>

Once the script completes, a number of outputs are created:

 - Several text files with information about the clustering
 - A number of plots in PDF format
 - A directory for each cluster (e.g. `cluster.0`, `cluster.1`, etc.)
   containing the requested localization densities (for each sample
   individually, and for the combined samples), and an RMF file for
   the cluster centroid.
   
The generated plots are shown below:

<img src="images/allplots.png" width=700px title="All output plots" />

## Discussion<a id="exhaustdiscuss"></a>

By examination of these plots, first we can see that a large number of small clusters were determined, rather than a small number of dominant clusters (bottom right plot). The model score (top left plot) also does not appear to have converged.

To compare the two score distributions (top right plot) we perform the Kolmogorov-Smirnov two-sample test and calculate a *p* value and effect size *D* (since even a tiny difference between two distributions can be significant if the samples are large). The score distributions are considered similar if the difference is not statistically significant (*p* value > 0.05) or if the difference is significant (*p* value < 0.05) but its magnitude is small (*D* < 0.30). In this case the two distributions are quite similar.

For the convergence test, we again calculate a similarly measure χ² and its effect size V, plus the fraction of models that are present in clusters containing at least 10 models (bottom left plot). Sampling precision is the clustering threshold where cluster population >= 80% while χ² < 0.05 and V >= 0.1 (green, red and blue dashed lines, respectively). It should be clear from this plot that χ² is not varying smoothly with threshold and so the determined precision (black dashed line) is unreliable.

The combination of these plots strongly suggests that our sampling is insufficient. Note also that while the score distributions and cluster populations for the two samples look similar, this is probably also an artefact of our under-sampling (remember that we only made a single run, so our two samples were generated by randomly breaking that one run into two samples; likely they are correlated to some degree).

The next step suggested by these results is to carry out multiple independent runs (this is more likely to ensure that the samples are uncorrelated, plus it is easier to implement multiple runs on a cluster than one long run). Each run should also likely be longer than the run analyzed here.

As an example of a good result, see similar plots for the recent modeling of the yeast nuclear pore complex; see
[Kim *et al.*, "Integrative Structure and Functional Anatomy of a Nuclear Pore Complex." Nature **555**, 475-482, 2018](https://www.ncbi.nlm.nih.gov/pubmed/29539637) for more details.

<img src="images/npc-exhaust.png" width=600px title="Sampling exhaustiveness of NPC" />

In this case we see good model convergence (top left), similar score distributions (top right), well behaved statistics giving a sampling precision of 9 Å (bottom left), and one dominant cluster (bottom right).

> In some cases our sampling may not converge even with longer sampling runs. This may be an indication of sample heterogeneity, requiring multi-state modeling. For example, if the sample contains a complex in both open and closed forms, or with multiple compositions then it may not be possible to build a single model that represents both forms. See [Molnar *et al.* "Cys-Scanning Disulfide Crosslinking and Bayesian Modeling Probe the Transmembrane Signaling Mechanism of the Histidine Kinase, PhoQ." Structure **22**, 1239-1251, 2014](https://www.ncbi.nlm.nih.gov/pubmed/25087511) or [Shi *et al.* "A strategy for dissecting the architectures of native macromolecular assemblies." Nat Methods **12**, 1135-8, 2015](https://www.ncbi.nlm.nih.gov/pubmed/26436480) for examples.

# Fit to data<a id="fitdata"></a>

Fit to data is generally calculated as the fraction of cross-links satisfied, the cross correlation coefficient for the fit between the model and experimental EM density, or a similar score specific to a given type of data (either that used in the modeling or excluded). For example, the recent model of the yeast nuclear pore complex fits well against the cryo-ET data used as input:

<img src="images/NPC-et-satisfy.png " width=800px title="NPC fit to input XL-MS data" />

It also fits well against composites determined by affinity purification, and SAXS data, neither of which were used in the modeling itself:

<img src="images/NPC-notused-satisfy.png " width=800px title="NPC fit to composite and SAXS data" />

# Resampling<a id="resampling"></a>

Resampling is not covered in this tutorial, but for an example see [Viswanath *et al.* "The molecular architecture of the yeast spindle pole body core determined by Bayesian integrative modeling." Mol Biol Cell. **7**, 3298-3314, 2017](https://www.ncbi.nlm.nih.gov/pubmed/28814505). (In this case jackknifing was done with the FRET data used in the modeling; see Figure S7.)

# Further reading<a id="further"></a>

Once analysis is complete and models are produced that completely sample the space at a reasonable precision, fit the input and unused data, and make biological sense, it is time to proceed to deposit them in the [PDB-Dev database](https://pdb-dev.wwpdb.org/). See the deposition tutorial in the `rnapoliii/deposition` directory for more details.